# MODIS16A2数据重采样

尝试对MODIS16A2数据进行重采样。这里使用广泛被采用的最大值合成法。

## 最大值合成法

记$\{PET_i\}$为一个栅格中的降雨量时间序列，则在该时间范围内，这一栅格中的降雨量值可以用序列中的最大值$\mathrm{max}(\{PET_i\})$代替。

$$ PET=\mathrm{max}(\{PET_i\}) $$

扩展到二维形式，使用循环解决
```python
import numpy as np
PET_sequence = np.array([[[PET_tij for j in range(0,j)]for i in range(0,i)]for t in range(0,t)])

PET = []
for t in range(0,t):
    PET_ij = []
    for i in range(0,i):
        PET_i = []
        for j in range(0,j):
            PET_std = max([PET_sequence[i][j] for t in range(0,t)])
            PET_i.append(PET_std)
        PET_ij.append(PET_i)
    PET.append(PET_ij)

PET = np.array(PET)
```
使用了四重嵌套循环进行合成，效率相当的低。
能否进行一些优化？

可以使用`xarray`库自带的方法取最大值。`numpy`好像也有?

比如

~~~python
import numpy as np
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 5]])
b = np.array([[4, 5, 1], [2, 3, 6], [7, 8, 9]])

np.amax([a, b], axis=0)
~~~


MODIS数据是8天一个样品，有必要根据日进行划分
根据时间间隔，将8天的采样频率平均划分给12个月，如下表：

| **Month** | **Start_date** | **End_date** | **Length** | **Start_date_filename** | **End_date_filename** |
|-----------|----------------|--------------|------------|-------------------------|------------------------|
| **1**     | 1              | 32           | 31         | 1                       | 25                     |
| **2**     | 33             | 56           | 23         | 33                      | 49                     |
| **3**     | 57             | 88           | 31         | 57                      | 81                     |
| **4**     | 89             | 120          | 31         | 89                      | 113                    |
| **5**     | 121            | 152          | 31         | 121                     | 145                    |
| **6**     | 153            | 184          | 31         | 153                     | 177                    |
| **7**     | 185            | 216          | 31         | 185                     | 209                    |
| **8**     | 209            | 240          | 31         | 209                     | 233                    |
| **9**     | 241            | 272          | 31         | 241                     | 265                    |


In [21]:
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr
import shapely
import rioxarray as rxr
import rasterio
from affine import Affine

定义生成文件名字典的函数

In [26]:
def generate_filename_map(year:str)->dict:
    
    def generate_filename(year:str,date:str)->str:
        preifx = 'MOD16A2GF.006_PET_500m_doy'
        suffix = '_aid0001.tif'
        return preifx+year+date+suffix
    
    def calc_days(start:int,end:int)->list:
        length = end-start
        counts = length//8
        days = []
        for i in range(counts):
            days.append(start+i*8)
        days.append(end)
        return days
    
    months = [1,2,3,4,5,6,7,8,9,10,11,12]
    months_start_date_lst = [1,33,57,89,121,153,185,209,241,273,305,337]
    months_end_date_lst = [25,49,81,113,145,177,209,233,265,297,329,361]

    filename_map = {}

    for i in range(len(months)):
        start = months_start_date_lst[i]
        end = months_end_date_lst[i]
        days = calc_days(start,end)
        filename_map[months[i]] = []
        for day in days:
            filename_map[months[i]].append(generate_filename(year,str(day).zfill(3)))

    return filename_map

分别读取每一年的对应数据，并且合成单独的文件

In [23]:
base_dir = '../datas/MODIS/MODIS16A2GF/PET/'